In [1]:
import torch
from torch.nn import functional as F

torch.manual_seed

<function torch.random.manual_seed(seed) -> torch._C.Generator>

In [2]:
B=2
T=4
H=8

In [3]:
q = torch.rand((B,T,H))
k = torch.rand((B,T,H))
v = torch.rand((B,T,H))
print(f'k={k}')
print(f'q={q}')


k=tensor([[[0.2019, 0.7361, 0.6917, 0.6697, 0.8816, 0.8778, 0.5624, 0.6087],
         [0.5709, 0.1287, 0.7291, 0.9036, 0.2750, 0.4504, 0.2268, 0.7222],
         [0.8807, 0.1956, 0.9130, 0.8347, 0.4794, 0.2915, 0.6711, 0.3530],
         [0.5590, 0.8649, 0.8610, 0.3103, 0.6929, 0.2848, 0.9577, 0.8428]],

        [[0.8761, 0.6484, 0.3507, 0.7117, 0.3469, 0.2147, 0.1144, 0.5859],
         [0.7059, 0.6850, 0.6872, 0.3744, 0.8800, 0.0075, 0.0179, 0.4977],
         [0.3337, 0.7432, 0.7731, 0.0651, 0.9830, 0.6445, 0.3597, 0.3247],
         [0.6530, 0.6468, 0.2476, 0.0635, 0.7316, 0.6886, 0.7945, 0.3710]]])
q=tensor([[[0.1978, 0.8019, 0.5979, 0.0059, 0.9746, 0.1800, 0.3264, 0.7955],
         [0.8896, 0.2573, 0.8801, 0.9443, 0.7829, 0.1123, 0.7084, 0.2479],
         [0.3589, 0.7758, 0.0647, 0.7540, 0.1285, 0.1595, 0.4071, 0.1371],
         [0.6121, 0.3702, 0.4932, 0.4994, 0.7678, 0.1572, 0.9590, 0.2818]],

        [[0.6384, 0.3892, 0.6029, 0.6903, 0.5343, 0.8014, 0.3787, 0.3668],
         [0.718

In [6]:
att = q @ k.mT  # B,T,H @ B,H,T ->  B,T,T
print(att.shape)
att

torch.Size([2, 4, 4])


tensor([[[2.7328, 1.6551, 1.9014, 3.0303],
         [2.9482, 2.6415, 3.3964, 3.2323],
         [1.7589, 1.3318, 1.5859, 1.8011],
         [2.5974, 1.9109, 2.6355, 2.9746]],

        [[2.1300, 2.0555, 2.3104, 2.2413],
         [2.3075, 2.0408, 1.9662, 1.9763],
         [1.2959, 1.1706, 1.3060, 1.0651],
         [1.2573, 1.5795, 2.0924, 1.8180]]])

In [7]:
hn=torch.sqrt(torch.tensor(H, dtype=torch.float32))
att = att / hn  # preserves variance of att head outputs to avoid softmax spiking at net init
att

tensor([[[0.9662, 0.5852, 0.6722, 1.0714],
         [1.0423, 0.9339, 1.2008, 1.1428],
         [0.6219, 0.4709, 0.5607, 0.6368],
         [0.9183, 0.6756, 0.9318, 1.0517]],

        [[0.7531, 0.7267, 0.8168, 0.7924],
         [0.8158, 0.7215, 0.6952, 0.6987],
         [0.4582, 0.4139, 0.4617, 0.3766],
         [0.4445, 0.5584, 0.7398, 0.6428]]])

In [8]:
# tril only for up to T (if T<=max block size)
tril = torch.tril(torch.ones(T, T))
att = att.masked_fill(tril[:T, :T] == 0, float('-inf'))
att

tensor([[[0.9662,   -inf,   -inf,   -inf],
         [1.0423, 0.9339,   -inf,   -inf],
         [0.6219, 0.4709, 0.5607,   -inf],
         [0.9183, 0.6756, 0.9318, 1.0517]],

        [[0.7531,   -inf,   -inf,   -inf],
         [0.8158, 0.7215,   -inf,   -inf],
         [0.4582, 0.4139, 0.4617,   -inf],
         [0.4445, 0.5584, 0.7398, 0.6428]]])

In [9]:
# incorrect
F.softmax(att, dim=1)

tensor([[[0.2673, 0.0000, 0.0000, 0.0000],
         [0.2885, 0.4164, 0.0000, 0.0000],
         [0.1894, 0.2620, 0.4083, 0.0000],
         [0.2548, 0.3216, 0.5917, 1.0000]],

        [[0.2822, 0.0000, 0.0000, 0.0000],
         [0.3005, 0.3869, 0.0000, 0.0000],
         [0.2101, 0.2844, 0.4309, 0.0000],
         [0.2073, 0.3287, 0.5691, 1.0000]]])

In [10]:
# correct
att = F.softmax(att, dim=-1)  # softmax along last dim
att

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.5271, 0.4729, 0.0000, 0.0000],
         [0.3571, 0.3070, 0.3359, 0.0000],
         [0.2538, 0.1991, 0.2572, 0.2900]],

        [[1.0000, 0.0000, 0.0000, 0.0000],
         [0.5236, 0.4764, 0.0000, 0.0000],
         [0.3378, 0.3232, 0.3390, 0.0000],
         [0.2135, 0.2393, 0.2869, 0.2603]]])

In [11]:
res = att @ v  # B,T,T @ B,T,H -> B,T,H
print(res.shape)
res

torch.Size([2, 4, 8])


tensor([[[0.2395, 0.8903, 0.8418, 0.2317, 0.7645, 0.7126, 0.1985, 0.0238],
         [0.4762, 0.8484, 0.7398, 0.1337, 0.8032, 0.8105, 0.5670, 0.4566],
         [0.5825, 0.6805, 0.7405, 0.3314, 0.8606, 0.7897, 0.4025, 0.3463],
         [0.5795, 0.6547, 0.5354, 0.3889, 0.7296, 0.6297, 0.2975, 0.3038]],

        [[0.9888, 0.7917, 0.0610, 0.9895, 0.4889, 0.1577, 0.7100, 0.1624],
         [0.8815, 0.7760, 0.2741, 0.6149, 0.3139, 0.4549, 0.4355, 0.4427],
         [0.8543, 0.5235, 0.2623, 0.4000, 0.4367, 0.4058, 0.5921, 0.4014],
         [0.8403, 0.4639, 0.4385, 0.4497, 0.4025, 0.5393, 0.6753, 0.3717]]])